<a href="https://colab.research.google.com/github/Jrodos12/Organizacion-de-Datos/blob/main/Pandas_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#La 3-upla de palabras más común en los comentarios de los logs (⭐⭐)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#La 3-upla de palabras más común en los comentarios de los logs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords') #Capaz antes de usar nltk les pide que descarguen estas cositas

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from string import punctuation
from nltk.corpus import stopwords
stopwords = set(stopwords.words() + list(punctuation + '«' + '»') )
#el df es muy grande.... observo las columnas un poco para ver como optimizar el espacio
#despues de revisar las columnas dado el problema a resolver, me quedo con lo que necesito para trabajar

In [ ]:
df_logs = pd.read_csv("drive/MyDrive/logs.csv", usecols = [5])

In [ ]:
df_logs

,comment
0,el contenido era: 'Si aprendés Pensamiento Cie...
1,el contenido era: '{{destruir}}en algun lugar ...
2,el contenido era: '#REDIRECT [[Flash]]'
3,Sin información de origen ni de licencia. Hay ...
4,Sustituida por [[commons:Image:Heckert_GNU.png]]
...,...
12188941,Aviso de pruebas a usuario tras reversiones co...
12188942,Página creada con «{{subst:Aviso prueba2|Luis ...
12188943,NaN
12188944,Página redirigida a [[Contenedor]]


Verifico existencia de nans

In [ ]:
(df_logs["comment"].isna()).sum()

6856012

Dropeo las filas con comentario nan, ya que no pueden aportar 3-upla (no tienen palabras dentro)

In [ ]:
df_logs.dropna(inplace=True)

Tokenizo el comentario con word_tokenize

In [ ]:
df_logs["comment"] = df_logs["comment"].map(lambda x: word_tokenize(x))
df_logs

,comment
0,"[el, contenido, era, :, 'Si, aprendés, Pensami..."
1,"[el, contenido, era, :, ', {, {, destruir, }, ..."
2,"[el, contenido, era, :, ', #, REDIRECT, [, [, ..."
3,"[Sin, información, de, origen, ni, de, licenci..."
4,"[Sustituida, por, [, [, commons, :, Image, :, ..."
...,...
12188932,"[Página, creada, con, «, {, {, subst, :, Aviso..."
12188937,"[Página, creada, con, «, {, {, subst, :, Aviso..."
12188941,"[Aviso, de, pruebas, a, usuario, tras, reversi..."
12188942,"[Página, creada, con, «, {, {, subst, :, Aviso..."


Ahora descarto las stopwords de cada comentario

In [ ]:
df_logs["comment"] = df_logs["comment"].map(lambda x: [palabra for palabra in x if not palabra.lower() in stopwords] )
df_logs

,comment
0,"[contenido, 'Si, aprendés, Pensamiento, Cientí..."
1,"[contenido, destruir, lugar, lejano, cerca, la..."
2,"[contenido, REDIRECT, Flash]"
3,"[información, origen, licencia, alternativas, ..."
4,"[Sustituida, commons, Image, Heckert_GNU.png]"
...,...
12188932,"[Página, creada, subst, Aviso, prueba2|18, sep..."
12188937,"[Página, creada, subst, Aviso, prueba2|Avanza,..."
12188941,"[Aviso, pruebas, usuario, tras, reversiones, c..."
12188942,"[Página, creada, subst, Aviso, prueba2|Luis, A..."


Calculo las triplas de cada comentario

In [ ]:
df_logs["comment"] = df_logs["comment"].map(lambda x: list(nltk.trigrams(x))  )
df_logs

,comment
0,"[(contenido, 'Si, aprendés), ('Si, aprendés, P..."
1,"[(contenido, destruir, lugar), (destruir, luga..."
2,"[(contenido, REDIRECT, Flash)]"
3,"[(información, origen, licencia), (origen, lic..."
4,"[(Sustituida, commons, Image), (commons, Image..."
...,...
12188932,"[(Página, creada, subst), (creada, subst, Avis..."
12188937,"[(Página, creada, subst), (creada, subst, Avis..."
12188941,"[(Aviso, pruebas, usuario), (pruebas, usuario,..."
12188942,"[(Página, creada, subst), (creada, subst, Avis..."


Cuento cual es la tripla mas repetida

In [ ]:
def contar_triplas(lista_triplas, dicc):
  for tripla in lista_triplas:
    dicc[tripla] = dicc.get(tripla, 0) + 1
  return

In [ ]:
dicc = {}
df_logs["comment"].map(lambda x: contar_triplas(x, dicc))
(max(dicc, key=dicc.get), max(dicc.values())  )

(('w', 'User', 'Axxgreazz/Monobook-Suite|monobook-suite'), 386281)

Resultado

In [ ]:
dicc[('w', 'User', 'Axxgreazz/Monobook-Suite|monobook-suite')]

386281